In [155]:
import pandas as pd
history = pd.read_csv("history_full_data.csv")

In [156]:
import re
def extract_year_price(row):
    string = {}
    price_string = row['cost_text'].replace(",","")
    price_year = re.findall(r'\d+[$]\d+', price_string)
    for each_year in price_year:
        year = re.search(r'\d+[$]', each_year).group(0).replace("$","")
        price = re.search(r'[$]\d+', each_year).group(0).replace("$","")
        if price and year:
            string[int(year)] = int(price)    
    return string

In [157]:
history.apply(lambda row:extract_year_price(row),axis=1)

0                               {2021: 500, 2020: 550}
1                                          {2021: 390}
2                                          {2021: 645}
3                                          {2020: 430}
4                               {2019: 420, 2017: 410}
                             ...                      
33857     {2022: 410, 2019: 310, 2017: 295, 2015: 260}
33858                                      {2022: 550}
33859                                   {2022: 820000}
33860    {2022: 480, 2016: 300, 2014: 1300, 2013: 300}
33861                                  {2022: 1395000}
Length: 33862, dtype: object

In [158]:
history["price_in_history"] = history.apply(lambda row:extract_year_price(row),axis=1)

In [159]:
# counting the # of rows
for year in range(2017,2022):
    count = 0
    for index in range(33862):
        if year in history.iloc[index]['price_in_history']:
            count += 1
    print(f'{year}:{count}')

2017:4469
2018:4911
2019:5460
2020:5747
2021:6903


In [160]:
history['year'] = history.apply(lambda row:list(row['price_in_history'].items()),axis=1)

In [161]:
history = history.explode('year')

In [162]:
history = history[history['price_in_history']!={}]

In [163]:
history["years"]= history.apply(lambda row:row['year'][0],axis=1)
history["price"]= history.apply(lambda row:row['year'][1],axis=1)

In [165]:
history = history.drop(['Unnamed: 0', 'cost_text','price_in_history','year'], axis=1)

In [166]:
# overview about spliting the rent of each year
history

,name,postcode,property_type,bed,bath,latitude,longitude,car,years,price
0,"609/1 BOUVERIE STREET, CARLTON SOUTH",3000,Category : Unit/apmt,Bed : 3,Bath : 2,-37.806100,144.96122,NaN,2021,500
0,"609/1 BOUVERIE STREET, CARLTON SOUTH",3000,Category : Unit/apmt,Bed : 3,Bath : 2,-37.806100,144.96122,NaN,2020,550
1,"507/22 ORR STREET, CARLTON SOUTH",3000,Category : Unit/apmt,Bed : 2,Bath : 2,-37.806340,144.96535,NaN,2021,390
2,"112/1 QUEENSBERRY STREET, CARLTON SOUTH",3000,NaN,Bed : 2,Bath : 2,-37.805410,144.96875,Car : 1,2021,645
3,"3203/488 SWANSTON STREET, CARLTON SOUTH",3000,NaN,Bed : 2,Bath : 1,-37.805580,144.96354,NaN,2020,430
...,...,...,...,...,...,...,...,...,...,...
33860,"119 TAMARA CRESCENT, INVERLOCH",3996,Category : House,Bed : 3,Bath : 2,-38.638762,145.70929,Car : 2,2022,480
33860,"119 TAMARA CRESCENT, INVERLOCH",3996,Category : House,Bed : 3,Bath : 2,-38.638762,145.70929,Car : 2,2016,300
33860,"119 TAMARA CRESCENT, INVERLOCH",3996,Category : House,Bed : 3,Bath : 2,-38.638762,145.70929,Car : 2,2014,1300
33860,"119 TAMARA CRESCENT, INVERLOCH",3996,Category : House,Bed : 3,Bath : 2,-38.638762,145.70929,Car : 2,2013,300
